# It is scratch for full instruction and test of Caser.

### DataModule

----

LightningDataModule for Santander dataset.

A DataModule implements 5 key methods:

- ``prepare_data`` (things to do on 1 GPU/TPU, not on every GPU/TPU in distributed mode)
- ``setup`` (things to do on every accelerator in distributed mode)
- ``train_dataloader`` (the training dataloader)
- ``val_dataloader`` (the validation dataloader(s))
- ``test_dataloader`` (the test dataloader(s))

This allows you to share a full dataset without explaining how to download,
split, transform and process the data.

Read the docs:
    https://pytorch-lightning.readthedocs.io/en/latest/extensions/datamodules.html

In [1]:
import warnings
warnings.filterwarnings('ignore')

from src.datamodules.caser_datamodule import SantanderDataModule
import numpy as np
import torch

In [2]:
San = SantanderDataModule(seed=25)

---

``SantanderDataModule.prepare_data()`` - preproces data if needed.


The process consists of two parts: creating subsample and making interactions data. First one allows us to choose the amount of users which will be in dataset. Second one transforms table data in special format suitable for Caser model.

In [3]:
San.prepare_data()

making data subsample...
you have took 2000 users from 601769 possible users
dataset reduced to 34000 entries
process done
making interactions data...
progress: 25.0 %
progress: 75.0 %
        TIMESTAMP  USER_ID  ITEM_ID
0      1422392400  1376263        1
1      1422392400   220061        3
2      1422392400   220061        7
3      1422392400   220061       11
4      1422392400   220061       20
...           ...      ...      ...
62492  1464382800  1196488        1
62493  1464382800  1196669        1
62494  1464382800  1193770        1
62495  1464382800  1114714       11
62496  1464382800  1180766        1

[62497 rows x 3 columns]
created file data\preprocessed\train_ver2.csv
process done
making data split...
process done


You need to make sure that ``data_test.csv``, ``data_train.csv`` and ``data_val.csv`` appered in ``data\preprocessed`` after this part.

---

``SantanderDataModule.setup()`` - Load data. Set variables: `self.data_train`, `self.data_val`, `self.data_test`.

This method is called by lightning when doing `trainer.fit()` and `trainer.test()`.

In [4]:
San.setup()

Train, validation and test datasets are ready for use!


---

As a result, we obtain pytorch dataloaders for train, validation and test.

In [5]:
train_DL = San.train_dataloader()

In [6]:
sequence, user, target, negative_sample = next(iter(train_DL))

print(sequence[0], user[0], target[0], negative_sample[0])

tensor([1, 1, 1, 1, 1]) tensor([1616]) tensor([1]) tensor([10,  2, 15])


---

### Training Caser

In [7]:
%run -i train.py +seed=25

[2023-11-14 15:17:34,477][src.utils][INFO] - Disabling python warnings! <config.ignore_warnings=True>
[2023-11-14 15:17:34,478][src.utils][INFO] - Printing config tree with Rich! <config.print_config=True>


CONFIG
├── datamodule
│   └── _target_: src.datamodules.caser_datamodule.SantanderDataModule                                             
│       data_dir: C:\Users\Skoltech\Desktop\RecSystemFinal\Caser/data/                                             
│       dataset_name: train_ver2.csv                                                                               
│       dataset_name_test: train_ver2.csv                                                                          
│       batch_size: 64                                                                                             
│       num_workers: 0                                                                                             
│       pin_memory: false                                                                                          
│       sample_size: 2000                                                                                          
│       min_data_points: 17                                                                                        
│       seed: 0                                                                                                    
│       sequence_length: 5                                                                                         
│       target_length: 1                                                                                           
│       neg_samples: 3                                                                                             
│                                                                                                                  
├── model
│   └── _target_: src.models.caser.CaserModel                                                                      
│       data_dir: C:\Users\Skoltech\Desktop\RecSystemFinal\Caser/data/                                             
│       dataset_name: train_ver2.csv                                                                               
│       dataset_name_test: train_ver2.csv                                                                          
│       L: 5                                                                                                       
│       d: 50                                                                                                      
│       nh: 16                                                                                                     
│       nv: 4                                                                                                      
│       drop: 0.5                                                                                                  
│       ac_conv: relu                                                                                              
│       ac_fc: relu                                                                                                
│       l2: 1.0e-06                                                                                                
│       learning_rate: 0.001                                                                                       
│                                                                                                                  
├── callbacks
│   └── model_checkpoint:                                                                                          
│         _target_: pytorch_lightning.callbacks.ModelCheckpoint                                                    
│         monitor: val/Recall@3                                                                                    
│         mode: max                                                                                                
│         save_top_k: 1                                                                                            
│         save_last: true                                                                                          
│         

CONFIG
+-- datamodule
|   `-- _target_: src.datamodules.caser_datamodule.SantanderDataModule                                             
|       data_dir: C:\Users\Skoltech\Desktop\RecSystemFinal\Caser/data/                                             
|       dataset_name: train_ver2.csv                                                                               
|       dataset_name_test: train_ver2.csv                                                                          
|       batch_size: 64                                                                                             
|       num_workers: 0                                                                                             
|       pin_memory: false                                                                                          
|       sample_size: 2000                                                                                          
|       min_data_points: 17                                                                                        
|       seed: 0                                                                                                    
|       sequence_length: 5                                                                                         
|       target_length: 1                                                                                           
|       neg_samples: 3                                                                                             
|                                                                                                                  
+-- model
|   `-- _target_: src.models.caser.CaserModel                                                                      
|       data_dir: C:\Users\Skoltech\Desktop\RecSystemFinal\Caser/data/                                             
|       dataset_name: train_ver2.csv                                                                               
|       dataset_name_test: train_ver2.csv                                                                          
|       L: 5                                                                                                       
|       d: 50                                                                                                      
|       nh: 16                                                                                                     
|       nv: 4                                                                                                      
|       drop: 0.5                                                                                                  
|       ac_conv: relu                                                                                              
|       ac_fc: relu                                                                                                
|       l2: 1.0e-06                                                                                                
|       learning_rate: 0.001                                                                                       
|                                                                                                                  
+-- callbacks
|   `-- model_checkpoint:                                                                                          
|         _target_: pytorch_lightning.callbacks.ModelCheckpoint                                                    
|         monitor: val/Recall@3                                                                                    
|         mode: max                                                                                                
|         save_top_k: 1                                                                                            
|         save_last: true                                                                                          
|         

Global seed set to 25


[2023-11-14 15:17:34,569][src.training_pipeline][INFO] - Instantiating datamodule <src.datamodules.caser_datamodule.SantanderDataModule>
[2023-11-14 15:17:34,573][src.training_pipeline][INFO] - Instantiating model <src.models.caser.CaserModel>
[2023-11-14 15:17:34,658][src.training_pipeline][INFO] - Instantiating callback <pytorch_lightning.callbacks.ModelCheckpoint>
[2023-11-14 15:17:34,666][src.training_pipeline][INFO] - Instantiating callback <pytorch_lightning.callbacks.EarlyStopping>
[2023-11-14 15:17:34,667][src.training_pipeline][INFO] - Instantiating callback <pytorch_lightning.callbacks.RichModelSummary>
[2023-11-14 15:17:34,669][src.training_pipeline][INFO] - Instantiating callback <pytorch_lightning.callbacks.RichProgressBar>
[2023-11-14 15:17:34,671][src.training_pipeline][INFO] - Instantiating logger <pytorch_lightning.loggers.comet.CometLogger>


...

preparing data has already done!


COMET WARNING: To get all data logged automatically, import comet_ml before the following modules: torch.
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/verdangeta/recsys/c55547cb2eb144dc85d36a85391df76b



┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│        test/MRR@1         │    0.8453661034306196     │
│        test/MRR@2         │    0.8428059395801332     │
│        test/MRR@3         │    0.8419667747624736     │
│        test/NDCG@1        │    0.8453661203384399     │
│        test/NDCG@2        │     0.82179194688797      │
│        test/NDCG@3        │    0.8243833184242249     │
│       test/Recall@1       │    0.8453661203384399     │
│       test/Recall@2       │    0.8166922926902771     │
│       test/Recall@3       │    0.8235193490982056     │
└───────────────────────────┴───────────────────────────┘

COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml ExistingExperiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/verdangeta/recsys/c55547cb2eb144dc85d36a85391df76b
COMET INFO:   Metrics:
COMET INFO:     test/MRR@1    : 0.8453661034306196
COMET INFO:     test/MRR@2    : 0.8428059395801332
COMET INFO:     test/MRR@3    : 0.8419667747624736
COMET INFO:     test/NDCG@1   : 0.8453661203384399
COMET INFO:     test/NDCG@2   : 0.82179194688797
COMET INFO:     test/NDCG@3   : 0.8243833184242249
COMET INFO:     test/Recall@1 : 0.8453661203384399
COMET INFO:     test/Recall@2 : 0.8166922926902771
COMET INFO:     test/Recall@3 : 0.8235193490982056
COMET INFO:   Others:
COMET INFO:     Created from : pytorch-lightning
COMET INFO:     Name      

[2023-11-14 15:20:43,584][src.training_pipeline][INFO] - Finalizing!
[2023-11-14 15:20:43,585][src.training_pipeline][INFO] - Best model ckpt at C:\Users\Skoltech\Desktop\RecSystemFinal\Caser\logs\experiments\runs\caser_santander_64\2023-11-14_15-17-34\checkpoints\epoch_001.ckpt


## Testing Caser

In [14]:
%run -i test.py +seed=25

[2023-11-14 15:26:09,011][src.utils][INFO] - Disabling python warnings! <config.ignore_warnings=True>

[2023-11-14 15:26:09,012][src.utils][INFO] - Printing config tree with Rich! <config.print_config=True>

CONFIG
├── datamodule
│   └── _target_: src.datamodules.caser_datamodule.SantanderDataModule                                             
│       data_dir: C:\Users\Skoltech\Desktop\RecSystemFinal\Caser/data/                                             
│       dataset_name: train_ver2.csv                                                                               
│       dataset_name_test: train_ver2.csv                                                                          
│       batch_size: 64                                                                                             
│       num_workers: 0                                                                                             
│       pin_memory: false                                                                                          
│       sample_size: 2000                                                                                          
│       min_data_points: 17                                                                                        
│       seed: 0                                                                                                    
│       sequence_length: 5                                                                                         
│       target_length: 1                                                                                           
│       neg_samples: 3                                                                                             
│                                                                                                                  
├── model
│   └── _target_: src.models.caser.CaserModel                                                                      
│       data_dir: C:\Users\Skoltech\Desktop\RecSystemFinal\Caser/data/                                             
│       dataset_name: train_ver2.csv                                                                               
│       dataset_name_test: train_ver2.csv                                                                          
│       L: 5                                                                                                       
│       d: 50                                                                                                      
│       nh: 16                                                                                                     
│       nv: 4                                                                                                      
│       drop: 0.5                                                                                                  
│       ac_conv: relu                                                                                              
│       ac_fc: relu                                                                                                
│       l2: 1.0e-06                                                                                                
│       learning_rate: 0.001                                                                                       
│                                                                                                                  
├── callbacks
│   └── model_checkpoint:                                                                                          
│         _target_: pytorch_lightning.callbacks.ModelCheckpoint                                                    
│         monitor: val/Recall@3                                                                                    
│         mode: max                                                                                                
│         save_top_k: 1                                                                                            
│         save_last: true                                                                                          
│         

CONFIG
+-- datamodule
|   `-- _target_: src.datamodules.caser_datamodule.SantanderDataModule                                             
|       data_dir: C:\Users\Skoltech\Desktop\RecSystemFinal\Caser/data/                                             
|       dataset_name: train_ver2.csv                                                                               
|       dataset_name_test: train_ver2.csv                                                                          
|       batch_size: 64                                                                                             
|       num_workers: 0                                                                                             
|       pin_memory: false                                                                                          
|       sample_size: 2000                                                                                          
|       min_data_points: 17                                                                                        
|       seed: 0                                                                                                    
|       sequence_length: 5                                                                                         
|       target_length: 1                                                                                           
|       neg_samples: 3                                                                                             
|                                                                                                                  
+-- model
|   `-- _target_: src.models.caser.CaserModel                                                                      
|       data_dir: C:\Users\Skoltech\Desktop\RecSystemFinal\Caser/data/                                             
|       dataset_name: train_ver2.csv                                                                               
|       dataset_name_test: train_ver2.csv                                                                          
|       L: 5                                                                                                       
|       d: 50                                                                                                      
|       nh: 16                                                                                                     
|       nv: 4                                                                                                      
|       drop: 0.5                                                                                                  
|       ac_conv: relu                                                                                              
|       ac_fc: relu                                                                                                
|       l2: 1.0e-06                                                                                                
|       learning_rate: 0.001                                                                                       
|                                                                                                                  
+-- callbacks
|   `-- model_checkpoint:                                                                                          
|         _target_: pytorch_lightning.callbacks.ModelCheckpoint                                                    
|         monitor: val/Recall@3                                                                                    
|         mode: max                                                                                                
|         save_top_k: 1                                                                                            
|         save_last: true                                                                                          
|         

Global seed set to 25


...

Train, validation and test datasets are ready for use!

Restoring states from the checkpoint path at C:\Users\Skoltech\Desktop\RecSystemFinal\Caser\test_ckpt\last.ckpt
Loaded model weights from the checkpoint at C:\Users\Skoltech\Desktop\RecSystemFinal\Caser\test_ckpt\last.ckpt


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│        test/MRR@1         │    0.8131080389144906     │
│        test/MRR@2         │    0.8269329237071172     │
│        test/MRR@3         │    0.8260937588894579     │
│        test/NDCG@1        │    0.8131080269813538     │
│        test/NDCG@2        │    0.7887366414070129     │
│        test/NDCG@3        │     0.791541337966919     │
│       test/Recall@1       │    0.8131080269813538     │
│       test/Recall@2       │    0.7869943380355835     │
│       test/Recall@3       │    0.7953575253486633     │
└───────────────────────────┴───────────────────────────┘

COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/verdangeta/recsys/e0ef6f8a21634228b332a9e82f9f1b2f
COMET INFO:   Metrics:
COMET INFO:     test/MRR@1    : 0.8131080389144906
COMET INFO:     test/MRR@2    : 0.8269329237071172
COMET INFO:     test/MRR@3    : 0.8260937588894579
COMET INFO:     test/NDCG@1   : 0.8131080269813538
COMET INFO:     test/NDCG@2   : 0.7887366414070129
COMET INFO:     test/NDCG@3   : 0.791541337966919
COMET INFO:     test/Recall@1 : 0.8131080269813538
COMET INFO:     test/Recall@2 : 0.7869943380355835
COMET INFO:     test/Recall@3 : 0.7953575253486633
COMET INFO:   Others:
COMET INFO:     Created from : pytorch-lightning
COMET INFO:     Name         : de

Model works well if all metrics are higher than 0.8

## Product recommendation for users

In [7]:
ckpt_path = "./logs/experiments/runs/caser_santander_64/2023-11-07_16-49-37/checkpoints/epoch_002.ckpt"

user_id = np.random.choice(train_DL.dataset.users.reshape(-1),2)

print(user_id)

[ 965 1460]


In [12]:
from inference_caser import make_predict

make_predict(train_DL, user_id,ckpt_path, k = 5)

Resume training from ./logs/experiments/runs/caser_santander_64/2023-11-07_16-49-37/checkpoints/epoch_002.ckpt
Predictions for users:
{965: tensor([ 4, 11, 10,  7,  5]), 1460: tensor([ 9,  4,  8, 10, 14])}
